In [ ]:
import sys
!{sys.executable} -m pip install rdflib

In [7]:
import sys
!{sys.executable} -m pip install pyyaml

In [9]:
from rdflib import Graph
from rdflib import URIRef

from rdflib.namespace import RDFS

In [21]:
#load graph
g = Graph()
g.parse("https://bioschemas.org/types/bioschemas_types.jsonld")

schema_domain = URIRef("https://schema.org/domainIncludes")
schema_range = URIRef("https://schema.org/rangeIncludes")
bs_ns = "https://bioschemas.org/terms/"

for s in g.subjects(predicate=schema_domain, unique=True) :
    prop_name = str(s).split('/')[-1]
    
    if bs_ns in s : #it is a bs property, we want to create a page for it
        prop_yaml_str = 'prop_name: ' + prop_name + '\n'
        prop_yaml_str +=  'redirect_from: ' + "/terms/" + prop_name + '\n'
        prop_yaml_str +=  'description: "' + g.value(s, RDFS.comment) + '"\n'

        all_domain = 'used_in_types: \n'
        for prop_domain in g.objects(subject=s, predicate=schema_domain) :
            dom_name = str(prop_domain).split('/')[-1]
            all_domain += '- ' + dom_name + '\n'

        all_range = 'expected_values_as_type: \n' 
        for prop_range in g.objects(subject=s, predicate=schema_range) :
            range_name = str(prop_range).split('/')[-1]
            all_range += '- ' + range_name + '\n'

        prop_yaml_str += all_domain
        prop_yaml_str += all_range
        with open(prop_name + '.html', 'w') as file:
            file.write("---\n")
            file.write(prop_yaml_str)
            file.write("\n---\n\n")
            file.write("{% include properties.html %}\n")
